# 마포구청 모범음식점 크롤링

In [ ]:
!pwd

In [ ]:
# urls
urls = ['https://www.mapo.go.kr/site/culture/mo/onstop_food_mo_list', '']
mapo_url = 'https://www.mapo.go.kr/site/culture/mo/onstop_food_mo_list'

In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time

마포구청 음식점 사이트 열기

In [ ]:
driver = webdriver.Chrome()
driver.get(mapo_url)

# implicitly wait for n seconds
driver.implicitly_wait(3)

# scroll down some
driver.execute_script("window.scrollTo(0, 1000)") 

class CoolFood:
    def __init__(self, r_id = -1, r_name = "", r_type = "", r_main_menu = "", r_addrs = "", r_dial_num= "", r_url = ""):
        self.r_id = r_id
        self.r_name = r_name
        self.r_type = r_type
        self.r_main_menu = r_main_menu
        self.r_addrs = r_addrs
        self.r_dial_num = r_dial_num
        self.r_url = r_url

### page 데이터 가져오기

In [ ]:
page_info = driver.find_elements(By.CSS_SELECTOR, ".bbs_paging a") # 맨처음/이전/숫자 1~10/다음/맨끝
page_info = page_info[2:]

In [ ]:
first_pages = len(page_info)
first_pages

In [ ]:
# page질

now = 0
prev_first_page = -1

# paginatoin flag
is_page_ended = False

while (is_page_ended == False):
    
    time.sleep(1.5)
    print("현재 페이지: " , now+1)
    



    if now == (len(page_info) - 2 -1):
        now = 0
        
        print("페이지 이동합니다!", page_info[0].text)
        prev_first_page = int(page_info[0].text)
        # print("이동 전 맨 첫페이지:", prev_first_page)
        
        # 다음페이지로 가기
        page_info[-2].click()
        time.sleep(1.5)
        
        # link 다시 받아오기
        page_info = driver.find_elements(By.CSS_SELECTOR, ".bbs_paging a") # 맨처음/이전/숫자 1~10/다음/맨끝
        page_info = page_info[2:]
        
        # 다음페이지의 맨 첫 아이템과 현 페이지 가장 첫 아이템이 같으면 플래그 풀어주기
        if int(page_info[0].text) == prev_first_page:
            is_page_ended = True
            # 중복된 정보 제거해주기
            pass
        
        
    else:
        page_info[now+1].click()
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, 900)") 
        time.sleep(1.5)
        now += 1

    # link 다시 받아오기
    page_info = driver.find_elements(By.CSS_SELECTOR, ".bbs_paging a") # 맨처음/이전/숫자 1~10/다음/맨끝
    page_info = page_info[2:]
    print("\t페이지수: ", len(page_info) - 2)
    
    
    # wait for full load
    time.sleep(1)
    
    # 음식점 정보 받아오는 부분
    stores = []
    
    # 한 페이지의 음식점 정보 한번에 row에 담기
    tbody = driver.find_element(By.CSS_SELECTOR, "tbody")
    rows = tbody.find_elements(By.CSS_SELECTOR, "tr")
    
    
    
    

#### 1 페이지

tbody = driver.find_element(By.CSS_SELECTOR, "tbody")
rows = tbody.find_elements(By.CSS_SELECTOR, "tr")

tr의 td 얻어오기

# rows[0].text.split()

상세보기 url 얻어오기

# rows[0].find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

full 음식점 객체

#### 한 페이지 음식점 리스트 받아오기

In [ ]:
# 담아올 객체
stores = []

In [ ]:
for row in rows:
    row_tds = row.find_elements(By.CSS_SELECTOR, 'td')
    record = [td.text for td in row_tds]
    try:
        record[-1] = row.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
    except:
        record[-1] = '-'
    stores.append(record)

In [ ]:
stores

### 페이지 이동

In [ ]:
now = 0+2

# go to next page
# if idx == len(page_info) - 2 - 1, go to next page


if now == (len(page_info) - 2 -1):
    is_page_ended = True
    pass
else:
    page_info[now].click()
    now += 1

# wait for full load
driver.implicitly_wait(2.5)

### 끝 페이지 도달할때까지 계속 다음페이지 누르기

## 데이터프레임 만들기

In [ ]:
# storeDf = pd.DataFrame(stores, columns = ['번호', '식당명', '유형', '주메뉴', '위치', '전화번호', '홈페이지'])
# storeDf

In [ ]:
# driver.quit()